In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 본 실습에서 사용할 데이터셋은 deepFM에서 사용한 데이터셋과 동일합니다!
# 단순히 벤치마크 데이터셋 그대로를 사용하기보다, 우리가 가지고있는 데이터셋을 모델 규격에 맞게 가공하여 입력으로 넣어봅시다.
# 1) SASRec은 시계열(시퀀스) 모델로, 데이터셋에서 각 유저(ip) 별 시청한 아이템(app)이 시간 순으로 정렬되어있음을 가정합니다.
# 2) 또한 모델 학습 및 추론 과정에서 사용되는 정보는 유저id와 아이템id가 유일합니다.
# 3) 마지막으로, SASRec은 특정 유저의 과거 선택 기록을 기반으로 현재 어떤 아이템을 선택할지 예측하는 모델이라는 점에서 양성(label=True) 데이터만 남기고 음성 데이터는 전부 지워줍니다.
# 위 3단계를 구현해봅시다.

df = pd.read_csv('./train_sample.csv')

# 양성 데이터만 선별
df = df.loc[df['is_attributed']==1]

# 유저 id와 아이템 id는 1부터 +1씩 증가하도록 변환해야합니다. 왜 이렇게 불편하게 설계했는지는 잘 모르겠지만.. 그걸 가정하니 규격에 맞게 바꿔줍시다.
for feat in ['ip', 'app']: 
    le = LabelEncoder() 
    df[feat] = le.fit_transform(df[feat])+1
    
# 유저 별, 시간 순 정렬
df = df.sort_values(by=['ip','click_time'])

# 필요한 정보(유저id, 아이템id)만 .txt(sep는 ','이 아니라 ' ' 입니다.)으로 저장
df[['ip', 'app']].to_csv('./data/movie.txt', sep=' ', header=False, index=False)
#df

In [4]:
import os
import time
import torch
from model import SASRec
from utils import *
import yaml
from box import Box



def str2bool(s):
    if s not in {"false", "true"}:
        raise ValueError("Not a valid boolean string")
    return s == "true"



conf_url = 'hyperparameters.yaml'
with open(conf_url, 'r') as f:
	config_yaml = yaml.load(f, Loader=yaml.FullLoader)
args = Box(config_yaml)



args.dataset = 'movie' ####### TODO: data 폴더 내부 txt파일명(suffix 제외) ex) ml-1m, Beauty 등. 우리는 movie.txt로 저장했으니 movie를 적어줍시다.
args.train_dir = 'test' ####### TODO: 모델 저장할 폴더명 아무거나 (앞으로 ./{dataset}_{train_dir} 폴더에 log, model 저장됨)



if not os.path.isdir(args.dataset + "_" + args.train_dir):
    os.makedirs(args.dataset + "_" + args.train_dir)
with open(os.path.join(args.dataset + "_" + args.train_dir, "args.txt"), "w") as f:
    f.write("\n".join([str(k) + "," + str(v) for k, v in sorted(vars(args).items(), key=lambda x: x[0])]))
f.close()

u2i_index, i2u_index = build_index(args.dataset)

# global dataset
dataset = data_partition(args.dataset)

[user_train, user_valid, user_test, usernum, itemnum] = dataset
# num_batch = len(user_train) // args.batch_size # tail? + ((len(user_train) % args.batch_size) != 0)
num_batch = (len(user_train) - 1) // args.batch_size + 1
cc = 0.0
for u in user_train:
    cc += len(user_train[u])
print("average sequence length: %.2f" % (cc / len(user_train)))

f = open(os.path.join(args.dataset + "_" + args.train_dir, "log.txt"), "w")
f.write("epoch (val_ndcg, val_hr) (test_ndcg, test_hr)\n")

sampler = WarpSampler(user_train, usernum, itemnum, batch_size=args.batch_size, maxlen=args.maxlen, n_workers=3)
model = SASRec(usernum, itemnum, args).to(args.device)  # no ReLU activation in original SASRec implementation?

for name, param in model.named_parameters():
    try:
        torch.nn.init.xavier_normal_(param.data)
    except:
        pass  # just ignore those failed init layers

model.pos_emb.weight.data[0, :] = 0
model.item_emb.weight.data[0, :] = 0

    

average sequence length: 1.62


Model training

In [5]:

model.train()  # enable model training

epoch_start_idx = 1

# ce_criterion = torch.nn.CrossEntropyLoss()
# https://github.com/NVIDIA/pix2pixHD/issues/9 how could an old bug appear again...
bce_criterion = torch.nn.BCEWithLogitsLoss()  # torch.nn.BCELoss()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, betas=(0.9, 0.98))

best_val_ndcg, best_val_hr = 0.0, 0.0
best_test_ndcg, best_test_hr = 0.0, 0.0
T = 0.0
t0 = time.time()
fname = f"SASRec_epoch{args.num_epochs}.pth"
folder = args.dataset + "_" + args.train_dir
save_path = ""


for epoch in range(epoch_start_idx, args.num_epochs + 1):
    for step in range(num_batch):  # tqdm(range(num_batch), total=num_batch, ncols=70, leave=False, unit='b'):

        u, seq, pos, neg = sampler.next_batch()  # tuples to ndarray
        u, seq, pos, neg = np.array(u), np.array(seq), np.array(pos), np.array(neg)
        pos_logits, neg_logits = model(u, seq, pos, neg)
        pos_labels, neg_labels = torch.ones(pos_logits.shape, device=args.device), torch.zeros(
            neg_logits.shape, device=args.device
        )
        # print("\neye ball check raw_logits:"); print(pos_logits); print(neg_logits) # check pos_logits > 0, neg_logits < 0
        adam_optimizer.zero_grad()
        indices = np.where(pos != 0)
        loss = bce_criterion(pos_logits[indices], pos_labels[indices])
        loss += bce_criterion(neg_logits[indices], neg_labels[indices])
        for param in model.item_emb.parameters():
            loss += args.l2_emb * torch.norm(param)
        loss.backward()
        adam_optimizer.step()
        
        if step%(num_batch//10) == 0:
            print(
                "loss in epoch {} iteration {}: {}".format(epoch, step, loss.item())
            )  # expected 0.4~0.6 after init few epochs

    if epoch % 1 == 0:
        model.eval()
        t1 = time.time() - t0
        T += t1
        print("Evaluating", end="")
        t_test = evaluate(model, dataset, args)
        t_valid = evaluate_valid(model, dataset, args)
        print(
            "epoch:%d, time: %f(s), valid (NDCG@10: %.4f, HR@10: %.4f), test (NDCG@10: %.4f, HR@10: %.4f)"
            % (epoch, T, t_valid[0], t_valid[1], t_test[0], t_test[1])
        )

        if (
            t_valid[0] > best_val_ndcg
            or t_valid[1] > best_val_hr
            or t_test[0] > best_test_ndcg
            or t_test[1] > best_test_hr
        ):
            best_val_ndcg = max(t_valid[0], best_val_ndcg)
            best_val_hr = max(t_valid[1], best_val_hr)
            best_test_ndcg = max(t_test[0], best_test_ndcg)
            best_test_hr = max(t_test[1], best_test_hr)
            folder = args.dataset + "_" + args.train_dir
            temp_fname = f"SASRec_epoch{epoch}.pth"
            save_path = os.path.join(folder, temp_fname)
            torch.save(model.state_dict(), save_path)

        f.write(str(epoch) + " " + str(t_valid) + " " + str(t_test) + "\n")
        f.flush()
        t0 = time.time()
        model.train()

    if epoch == args.num_epochs:
        save_path = os.path.join(folder, fname)
        torch.save(model.state_dict(), save_path)

f.close()
sampler.close()
print("Done")


loss in epoch 1 iteration 0: 1.271963119506836
loss in epoch 1 iteration 191: 0.3266589045524597
loss in epoch 1 iteration 382: 0.35938864946365356
loss in epoch 1 iteration 573: 0.3367638885974884
loss in epoch 1 iteration 764: 0.32608458399772644
loss in epoch 1 iteration 955: 0.31127694249153137
loss in epoch 1 iteration 1146: 0.3039439022541046
loss in epoch 1 iteration 1337: 0.387241393327713
loss in epoch 1 iteration 1528: 0.26362931728363037
loss in epoch 1 iteration 1719: 0.300498366355896
loss in epoch 1 iteration 1910: 0.3208335041999817
Evaluating......................epoch:1, time: 32.614681(s), valid (NDCG@10: 0.8462, HR@10: 0.9563), test (NDCG@10: 0.8406, HR@10: 0.9662)
loss in epoch 2 iteration 0: 0.2981089949607849
loss in epoch 2 iteration 191: 0.34493815898895264
loss in epoch 2 iteration 382: 0.332329660654068
loss in epoch 2 iteration 573: 0.29634028673171997
loss in epoch 2 iteration 764: 0.23454266786575317
loss in epoch 2 iteration 955: 0.2861711382865906
loss in

Model test

In [6]:
args.state_dict_path = save_path
model = SASRec(usernum, itemnum, args).to(args.device)  # no ReLU activation in original SASRec implementation?
model.load_state_dict(torch.load(args.state_dict_path, map_location=torch.device(args.device)))

model.eval()
t_test = evaluate(model, dataset, args)
print("test (NDCG@10: %.4f, HR@10: %.4f)" % (t_test[0], t_test[1]))

C:\Users\USER\AppData\Local\Temp\ipykernel_15732\587850054.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.state_dict_path, map_loc

............test (NDCG@10: 0.8442, HR@10: 0.9521)


[Quest]

1) squence model의 train_test_split method

train 단계와 test 단계에서 데이터를 어떻게 구분하는지 고려해봐야합니다.

만약 100명의 유저가 있다고 하면, 80:20으로 구분하여 train_test_split을 했을까요?
그렇다면 train단계에서 학습되지 못한 20명의 유저 정보를 추론할 때 정확도가 높지 않을 확률이 높습니다.
해당 모델은 시퀀스 모델이기에 일반적인 피처 예측 모델과는 꽤 다른 방식으로 train, valid, test 데이터를 구분합니다. 
소스코드를 살펴보면서 어떤 방식으로 train_test_split을 하는지 기술해봅시다. 왜 해당 방식으로 split하는 게 합당한지 본인의 생각을 적어주세요.

(힌트: util.py의 data_partition, evaluate 함수를 살펴봅시다. torch, numpy를 잘 몰라 이해가 잘 안된다면 주석을 참고해서 작동 방식을 추측해주셔도 좋습니다!)

답변: 



2) NDCG, HR 성능평가지표의 의미

 본 코드에서는 SASRec 모델의 성능 지표로 NDCG@10, HR@10을 사용하고 있습니다. 
작성자의 경우, NDCG@10은 0.84, HR@10은 0.95 정도가 나오는데요. 


2-1) HR@10이 0.95가 나온다는 게 무슨 의미인지 HR@10의 정의를 고려하여 설명해봅시다.

답변: 


2-2) NDCG@10=0.88, HR@10=0.9 가 나왔다고 가정해봅시다. 이 두 지표가 유사한 값을 가진다는 게 무슨 의미인지 NDCG@10의 정의를 고려하여 설명해봅시다.

답변:







3) deepFM과 SASRec의 차이

deepFM, SASRec 두 가지 딥러닝 기반 추천시스템 모델을 사용해보았습니다. 
과연 두 모델 중 어떤 모델이 특정 상황에서 더 적합할지 생각해봐야 합니다.

다음 상황을 가정해봅시다. 그핵이는 이커머스 플랫폼 GH팡에서 소비자들에게 상품을 추천해주는 추천시스템을 만들고 있습니다.
그러나 그핵이가 마주한 문제점은 대부분의 소비자들은 과거 구매 데이터가 거의 없어서 무엇을 좋아할지 예측하기 힘들다는 점이었습니다.
해당 비활성 고객으로부터 알아낼 수 있는 유일한 정보는 연령, 성별, 관심카테고리 등 서비스 회원가입을 할 때 필수적으로 기입해야하는 정보 뿐입니다.
이렇게 활성 고객과 비활성 고객이 섞여있는 유저 집단에 맞춤형 추천을 하기 위해서 deepFM과 SASRec을 어떻게 활용하면 좋을지 본인의 생각에 따라 추천시스템 구조를 간단하게 설계하고 그 근거를 말해봅시다.

(생각해볼 포인트: 활성 정도에 따라 고객을 클러스터링하는 데 성공했다면, 클러스터 별로 다른 모델을 적용할 수도 있습니다. 어떤 군집 패턴을 분석할 때 어떤 모델에 더 적합할지 생각해봅시다.)

답변: